In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
from pprint import pprint
import json
from geojson import Feature, FeatureCollection, Point

In [ ]:
response = requests.get("https://api.openbrewerydb.org/breweries?page=402")
data = response.json()

In [ ]:
pprint(data[0])

In [ ]:
for brewery in data[0]:
    pprint(data)

In [2]:
breweries_df = pd.read_csv('breweries.csv')
breweries_df.head(3)

,id,name,brewery_type,street,city,state,postal_code,website_url,phone,created_at,updated_at,country,longitude,latitude,tags
0,siluria-brewing-company,Siluria Brewing Company,planning,NaN,Alabaster,Alabama,35007-8501,http://www.siluriabrewing.com,2.054828e+09,32:47.7,20:27.0,United States,-86.816377,33.244281,NaN
1,cheaha-brewing-co,Cheaha Brewing Co,brewpub,1208 Walnut Ave,Anniston,Alabama,36201-4526,http://www.cheahabrewingcompany.com,2.567707e+09,32:47.4,20:08.8,United States,-85.833774,33.660167,NaN
2,avondale-brewing-co,Avondale Brewing Co,micro,201 41st St S,Birmingham,Alabama,35222-1932,http://www.avondalebrewing.com,2.057775e+09,32:47.3,19:57.8,United States,-86.774322,33.524521,NaN


In [6]:
drop_tags = breweries_df.drop(['tags'], axis=1)

In [7]:
brew_dropna = drop_tags.dropna(how='any')
brew_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3813 entries, 1 to 7920
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            3813 non-null   object 
 1   name          3813 non-null   object 
 2   brewery_type  3813 non-null   object 
 3   street        3813 non-null   object 
 4   city          3813 non-null   object 
 5   state         3813 non-null   object 
 6   postal_code   3813 non-null   object 
 7   website_url   3813 non-null   object 
 8   phone         3813 non-null   float64
 9   created_at    3813 non-null   object 
 10  updated_at    3813 non-null   object 
 11  country       3813 non-null   object 
 12  longitude     3813 non-null   float64
 13  latitude      3813 non-null   float64
dtypes: float64(3), object(11)
memory usage: 446.8+ KB


In [8]:
brew_dropna

,id,name,brewery_type,street,city,state,postal_code,website_url,phone,created_at,updated_at,country,longitude,latitude
1,cheaha-brewing-co,Cheaha Brewing Co,brewpub,1208 Walnut Ave,Anniston,Alabama,36201-4526,http://www.cheahabrewingcompany.com,2.567707e+09,32:47.4,20:08.8,United States,-85.833774,33.660167
2,avondale-brewing-co,Avondale Brewing Co,micro,201 41st St S,Birmingham,Alabama,35222-1932,http://www.avondalebrewing.com,2.057775e+09,32:47.3,19:57.8,United States,-86.774322,33.524521
3,trim-tab-brewing,Trim Tab Brewing,micro,2721 5th Ave S,Birmingham,Alabama,35233-3401,http://www.trimtabbrewing.com,2.057031e+09,32:47.8,20:31.4,United States,-86.791400,33.512849
8,ghost-train-brewing-co,Ghost Train Brewing Co,micro,2616 3rd Ave S,Birmingham,Alabama,35233-2619,http://www.ghosttrainbrewing.com,2.053702e+09,32:47.5,20:14.7,United States,-86.794062,33.514159
9,good-people-brewing-co,Good People Brewing Co,micro,114 14th St S,Birmingham,Alabama,35233-1415,http://www.goodpeoplebrewing.com,6.154984e+09,32:47.6,20:16.5,United States,-86.812180,33.507595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7910,coal-creek-tap,Coal Creek TAP,micro,108 E Grand Ave,Laramie,Wyoming,82070-3638,http://www.coalcreektap.com,3.074610e+09,34:37.4,46:42.2,United States,-105.595218,41.311056
7911,library-sports-grill-and-brewery,Library Sports Grill and Brewery,brewpub,201 E Custer St,Laramie,Wyoming,82070-3633,http://laramie.librarysportsgrille.com,3.077420e+09,34:37.5,46:52.7,United States,-105.594712,41.309384
7913,wyold-west-brewing-company,WYOld West Brewing Company,brewpub,221 N Bent St,Powell,Wyoming,82435-2335,http://www.wyoldwest.com,3.077646e+09,34:37.7,47:12.2,United States,-108.757481,44.754520
7915,bitter-creek-brewing-co,Bitter Creek Brewing Co,brewpub,604 Broadway St,Rock Springs,Wyoming,82901-6348,http://www.bittercreekbrewing.com,3.073625e+09,34:37.3,46:39.1,United States,-109.218124,41.586839


In [9]:
# save the Geometry as feature
features = breweries_df.apply(
    lambda row: Feature(geometry=Point((float(row['longitude']), float(row['latitude'])))),
    axis=1).tolist()

# all the other columns used as properties
properties = breweries_df.drop(['longitude', 'latitude'], axis=1).to_dict('records')

# whole geojson object
feature_collection = FeatureCollection(features=features, properties=properties)

In [10]:
with open('breweries.geojson', 'w', encoding='utf-8') as f:
    json.dump(feature_collection, f, ensure_ascii=False)